<a href="https://colab.research.google.com/github/vccheng2001/Apnea-Detection-LSTM/blob/master/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Your Own Optimizers
Optimizers tie the loss function and model parameters together by updating the model in response to the output of the loss function. In simpler terms, optimizers shape your model into its most accurate possible form by futzing with the weights. In even more simpler terms, optimizers define the way you want your model to be trained. 

The selection of optimizers could have dramatic effect on the performance of your model. If you choose a complex optimizer for a simple model, you would be likely to spend much more time in training with barely improved accuracy. And if you choose a naive optimizer for a complicated model, you would probably end up with poor accuracy.  

This exercise will cover:
- *Part 1: Calling existing optimizers provided by tf.keras*
- *Part 2: Implementing and evaluating your own optimizers*


## Part 1: Calling existing optimizers provided by tf.keras
In this part, we will build up a demo pipeline, demonstrating how to involve optimizers given by `tf.keras` into the training process. 

## (1a) Setting up the environment
Just run the following cell to establish the runtime environment. Note that we're using Tensorflow 2.0 for this part of assignment. 

In [76]:
from __future__ import absolute_import, division, print_function, unicode_literals

# If running locally, comment out the following try-except block. 
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
from tensorflow.python.ops import math_ops

## (1b) Building a baseline model
In this task, you will create a simple Linear Regression model with a synthetic toy dataset. Mean Squared Error (MSE) will be used as the loss metric. 

Note we have set a seed for `tf.random` for auto grading purpose. Please don't change the seed. 

In [75]:
# A toy dataset of points around y = 4 * x_1 + 3 * x_2 + 2
NUM_EXAMPLES = 2000
tf.random.set_seed(10605)   # DON'T CHANGE THIS!
X_train = tf.random.normal([NUM_EXAMPLES, 2])
noise = tf.random.normal([NUM_EXAMPLES, 1])
y_train = tf.matmul(X_train, [[4.], [3.]]) + 2 + noise

# Definition of the Linear Regression model
class Linear(tf.keras.Model):
  def __init__(self):
    super(Linear, self).__init__()
    self.w = tf.Variable([[5., 5.]], name='weight')
    self.b = tf.Variable(10., name='bias')

  # Implementation of Linear Regression
  # TODO: Replace <FILL IN> with appropriate code
  def call(self, inputs):
    """
      Implementation of Linear Regression. This function will use current parameters to predict the labels for inputs

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          inputs (tf.Tensor): The input tensor of this Linear Regression model.

      Returns:
          tf.Tensor: Predicted label
    """

    y_pred = tf.add(tf.matmul(inputs, self.w, transpose_b=True), self.b)
    return y_pred

# MSE loss function
# TODO: Replace <FILL IN> with appropriate code
def loss(y_true, y_pred):
  """
    Calculate MSE loss between the predicted label and the actual label.

    Note:
        You should try to make use of Tensorflow library to perform related calculation.

    Args:
        y_true (tf.Tensor): The actual label.
        y_pred (tf.Tensor): The predicted label.

    Returns:
        tf.Tensor: MSE loss value
  """
  mse_loss = tf.reduce_mean(tf.square(y_true - y_pred),keepdims=True)
  return mse_loss

In [71]:
# Test for LR_MSE


m, x, y = Linear(), tf.constant([[1.0, 2.0]]), tf.constant([3.0])
assert np.equal(m(x).numpy()[0][0], 25.0), "Wrong implementation of Linear Regression"
assert np.equal(loss(y,m(x)).numpy(), 484.0), "Wrong implementation of MSE loss"


## (1c) Applying an existing optimizer
Next, we will apply an Adam optimizer provided by `tf.keras` to our previously defined Linear Regression model. You can reuse this code snippet to validate your own optimizers.


In [72]:
model = Linear()
# Standard method to compile a tf.keras.Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss=loss)
# Standard method to fit a tf.keras.Model
print(f"X_train shape: {tf.shape(X_train)}")
print(f"y_train shape: {tf.shape(y_train)}")

model.fit(X_train, y_train, batch_size=32, epochs=10)
print(f"model.w shape: {model.w.shape}")


X_train shape: [2000    2]
y_train shape: [2000    1]
Epoch 1/10
63/63 [==============================] - 0s 777us/step - loss: 46.4401
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 4.6619
Epoch 3/10
63/63 [==============================] - 0s 794us/step - loss: 1.0225
Epoch 4/10
63/63 [==============================] - 0s 735us/step - loss: 1.0149
Epoch 5/10
63/63 [==============================] - 0s 951us/step - loss: 0.9971
Epoch 6/10
63/63 [==============================] - 0s 739us/step - loss: 0.9725
Epoch 7/10
63/63 [==============================] - 0s 726us/step - loss: 0.9351
Epoch 8/10
63/63 [==============================] - 0s 906us/step - loss: 0.9766
Epoch 9/10
63/63 [==============================] - 0s 783us/step - loss: 1.0021
Epoch 10/10
63/63 [==============================] - 0s 786us/step - loss: 0.9755
model.w shape: (1, 2)


## Part 2: Implementing your own optimizers
In this part, you will implement four different optimizers by yourself! Don't be panic if you're not good at math stuff, we will give you enough instructions and explanations!

Generally, an optimizer class should contain three methods:
- `__init__`: Initializes required parameters. 
-  `calculate_gradient`: Calculates gradients in the computational graph. 
- `apply_gradient`: Updates parameters of the model. 

Some of you may notice that we're not using the "keras-style" to implement our optimizers. Since keras is a high-level encapsulation, its implementation of optimizers is generally too abstract for you to get the mathematical essence. This hacky way can enable you to design your own optimizers in a lower, Tensorflow level.

## (2a) Gradient Descent
Let's start with the simplest Gradient Descent. 

For a Linear Regression model with MSE where $\mathbf{y}=w^T\mathbf{X} + b$, the update rules of parameters at the $t$ th epoch are as follows:

$$w^{(t+1)}=w^{(t)}-\alpha \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}) \mathbf{X}^{(i)}$$

$$b^{(t+1)}=b^{(t)}-\alpha \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}))$$

where $n$ is the size of input dataset. 



In [81]:
# TODO: Replace <FILL IN> with appropriate code
class GD():
  def __init__(self, lr=0.01):
    """Initializes required parameters.

      Args:
          lr (float): Learning rate.
    """
    self.lr = lr 

  def calculate_gradient(self, X, y, model):
      
   
    N = len(X)
    diff = model(X) - y
   
    grads_w = 2/N *  (diff*X)
    grads_b =  2/N * diff

    print("grads_w: ", tf.shape(grads_w))
    print("grads_b: ", tf.shape(grads_b))

    return grads_w, grads_b


  def apply_gradient(self, grads, model):
 
    print('reducegrad0 shape', tf.reduce_sum(grads[0],axis=0,keepdims=True))
    model.w = model.w - self.lr * tf.reduce_sum(grads[0],axis=0,keepdims=True)
    model.b = model.b - self.lr  * tf.reduce_sum(grads[1],axis=0,keepdims=True)


In [82]:
# Evaluation
model_0 = Linear()
opt_0 = GD()
print(loss(model_0(X_train), y_train).shape)

print("Initial loss: %.3f" % loss(model_0(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_0.calculate_gradient(X_train, y_train, model_0)
  # Use the optimizer to update gradients
  opt_0.apply_gradient(grads, model_0)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_0(X_train), y_train)))

print("Final loss: %.3f" % loss(model_0(X_train), y_train))
print("w = %s, b = %s" % (model_0.w.numpy(), model_0.b.numpy()))

(1, 1)
Initial loss: 68.725
grads_w:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
grads_b:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
reducegrad0 shape tf.Tensor([[2.06868   3.6274102]], shape=(1, 2), dtype=float32)
Loss at epoch 0: 66.072
grads_w:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
grads_b:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
reducegrad0 shape tf.Tensor([[2.0260062 3.5605822]], shape=(1, 2), dtype=float32)
grads_w:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
grads_b:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
reducegrad0 shape tf.Tensor([[1.9842119 3.4949636]], shape=(1, 2), dtype=float32)
grads_w:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
grads_b:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
reducegrad0 shape tf.Tensor([[1.9432728 3.4305403]], shape=(1, 2), dtype=float32)
grads_w:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
grads_b:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
reducegrad0 shape tf.Tensor([[1.9031719 

In [83]:
# Test case
assert np.allclose(loss(model_0(X_train), y_train), 0.969, atol=1e-3), 'Wrong implementation of GD'

## (2b) Stochastic Gradient Descent
Let's take one step further. In SGD, you should randomly choose one data point to calculate the gradient each time. 

In [86]:
# TODO: Replace <FILL IN> with appropriate code
class SGD():
  def __init__(self, lr=0.01):
    """Initializes required parameters.

      Args:
          lr (float): Learning rate.
    """
    # self.lr = <FILL IN>

    self.lr = lr 

  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (tf.Tensor): The input.
          y (tf.Tensor): The actual label.
          model (tf.keras.Model): The model used to predict y with given input X. 

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """
    import random 

    N = len(y)
    # randomly choose a training example i
    i = random.randint(0, N-1)
    xi = tf.expand_dims(X[i,:], axis=0)
    yi = y[i,:]

    diff = tf.subtract(model(xi), yi)
    # print('diff shape', diff.shape)
    # print('xi shape', xi.shape) # diff:11, xi:  12 -> 12
    # print('w shape', model.w.shape)
    grads_w =  2 * (tf.matmul(diff, xi, transpose_a=True))
    grads_b =  2 * (diff)
    return (grads_w, grads_b)

  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t W, gradient w.r.t B).
          model (tf.keras.Model): The model used to predict y with given input X. 
    """
    model.w = model.w - self.lr * grads[0]
    model.b = model.b - self.lr * grads[1]


In [87]:
# Evaluation
model_1 = Linear()
opt_1 = SGD()

print("Initial loss: %.3f" % loss(model_1(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_1.calculate_gradient(X_train, y_train, model_1)
  # Use the optimizer to update gradients
  opt_1.apply_gradient(grads, model_1)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_1(X_train), y_train)))

print("Final loss: %.3f" % loss(model_1(X_train), y_train))
print("w = %s, b = %s" % (model_1.w.numpy(), model_1.b.numpy()))

Initial loss: 68.725
Loss at epoch 0: 65.545
Loss at epoch 20: 34.270
Loss at epoch 40: 15.212
Loss at epoch 60: 6.076
Loss at epoch 80: 3.472
Loss at epoch 100: 2.323
Loss at epoch 120: 1.638
Loss at epoch 140: 1.312
Loss at epoch 160: 1.169
Loss at epoch 180: 1.169
Loss at epoch 200: 1.091
Loss at epoch 220: 1.021
Loss at epoch 240: 1.070
Loss at epoch 260: 1.071
Loss at epoch 280: 0.985
Final loss: 0.995
w = [[4.0214953 3.0479925]], b = [[2.1778488]]


In [88]:
# Test case
assert np.allclose(loss(model_1(X_train), y_train), 1.0, atol=0.05), 'Wrong implementation of SGD'

## (2c) AdaGrad

One of the shortcomings about Gradient Descent is that its performance highly depends on the initial learning rate. Poor selection of initial learning rate would lead to either slow convergence or incapability of finding local minimum. In order to solve this problem, a revised version called AdaGrad is introduced. 

As its name suggests, AdaGrad makes its learning rate adaptive. To be more specific, AdaGrad uses past squared gradients to form an accumulated regularizer for its learning rate. 

Assuming $g^{(t)}$ is the gradient at $t$ th epoch, AdaGrad first calculates the accumulated squared gradients:

$$r^{(t)}=r^{(t-1)}+(g^{(t)})^2$$

Then AdaGrad will use this accumulated squared gradients to regularize its learning rate:

$$w^{(t+1)}=w^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_w^{(t)}$$
$$b^{(t+1)}=b^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_b^{(t)}$$

where $\epsilon$ is an additive constant (usually set as $10^{-7}$) that ensures we do not divide by 0. 

Note that you can reuse the `calculate_gradient` function in SGD for AdaGrad. 


In [143]:
class AdaGrad():
  def __init__(self, lr=0.001, epsilon=1e-7):
    """Initializes required parameters.

      Note: self.accumulator is the accumulated squared gradients. It is a tuple with following format:
        (accumulator w.r.t w, accumulator w.r.t b)
        And it should be initialized with the value of 0.1

      Args:
          lr (float): Learning rate.
          epsilon (float): Additive constant that ensures we do not divide by 0.
    """
    self.lr = lr
    self.epsilon = epsilon
    self.accumulator = (0.1, 0.1)

    
  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (tf.Tensor): The input.
          y (tf.Tensor): The actual label.
          model (tf.keras.Model): The model used to predict y with given input X. 

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """
    N = len(X)

    diff = tf.subtract(model(X), y)
    grads_w =  tf.reduce_sum(2/N * (diff*X))
    grads_b =  tf.reduce_sum(2/N * (diff))
    print('calculated grads_w', tf.shape(grads_w))
    print('calculated grads_b', tf.shape(grads_b))

    print('end calc')
    return (grads_w, grads_b)
    


  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
          model (tf.keras.Model): The model used to predict y with given input X. 
    """
    
    self.accumulator =  (self.accumulator[0] + tf.square(grads[0]), self.accumulator[1] + tf.square(grads[1])) 

    r = self.accumulator
    model.w = model.w - (self.lr / (np.sqrt(r[0]) + self.epsilon)) * grads[0]
    model.b = model.b - (self.lr / (np.sqrt(r[1]) + self.epsilon)) * grads[1]
    print('grads0', tf.shape(grads[0]))
    print('grads0squ', tf.shape(tf.square(grads[0])))


    print("end apply")
    print('self.accum', self.accumulator)


In [144]:
# Evaluation
model_2 = Linear()
opt_2 = AdaGrad(lr=0.1)

print("Initial loss: %.3f" % loss(model_2(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_2.calculate_gradient(X_train, y_train, model_2)
  # Use the optimizer to update gradients
  opt_2.apply_gradient(grads, model_2)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_2(X_train), y_train)))

print("Final loss: %.3f" % loss(model_2(X_train), y_train))
print("w = %s, b = %s" % (model_2.w.numpy(), model_2.b.numpy()))

Initial loss: 68.725
calculated grads_w tf.Tensor([], shape=(0,), dtype=int32)
calculated grads_b tf.Tensor([], shape=(0,), dtype=int32)
end calc
grads0 tf.Tensor([], shape=(0,), dtype=int32)
grads0squ tf.Tensor([], shape=(0,), dtype=int32)
end apply
self.accum (<tf.Tensor: shape=(), dtype=float32, numpy=32.54546>, <tf.Tensor: shape=(), dtype=float32, numpy=250.69766>)
Loss at epoch 0: 66.604
calculated grads_w tf.Tensor([], shape=(0,), dtype=int32)
calculated grads_b tf.Tensor([], shape=(0,), dtype=int32)
end calc
grads0 tf.Tensor([], shape=(0,), dtype=int32)
grads0squ tf.Tensor([], shape=(0,), dtype=int32)
end apply
self.accum (<tf.Tensor: shape=(), dtype=float32, numpy=60.50036>, <tf.Tensor: shape=(), dtype=float32, numpy=495.21298>)
calculated grads_w tf.Tensor([], shape=(0,), dtype=int32)
calculated grads_b tf.Tensor([], shape=(0,), dtype=int32)
end calc
grads0 tf.Tensor([], shape=(0,), dtype=int32)
grads0squ tf.Tensor([], shape=(0,), dtype=int32)
end apply
self.accum (<tf.Tensor:

In [141]:
# Test case
assert np.allclose(loss(model_2(X_train), y_train), 29.0, atol=2.0), 'Wrong implementation of AdaGrad'


AssertionError: ignored

## (2d) Adam
Although it sounds like a powerful algorithm, AdaGrad still has a bunch of shortcomings. One of them is that as the accumulated squared gradients becoming larger, the learning rate will be pushed to 0, leading to an early stop. 

Adam is designed to improve AdaGrad. It relies on adaptive moment estimation to set an independent adaptive learning rate for each parameter. The idea of Adam at the $t$ th epoch is described below:
- Calculate current gradient $g^{(t)}$
- Update biased first moment estimate
  $$m^{(t)} = \beta_1\cdot m^{(t-1)}+(1-\beta_1)\cdot g^{(t)}$$
- Update biased second raw moment estimate
  $$v^{(t)} = \beta_2\cdot v^{(t-1)}+(1-\beta_2)\cdot (g^{(t)})^2$$
- Compute bias-corrected first moment estimate
  $$\hat{m}^{(t)} = \frac{m^{(t)}}{1-\beta_1^t}$$
- Compute bias-corrected second raw moment estimate
  $$\hat{v}^{(t)} = \frac{v^{(t)}}{1-\beta_2^t}$$
- Update parameters with constrained learning rate
  $$w^{(t)} = w^{(t-1)}-\alpha\cdot\frac{\hat{m}_w^{(t)}}{(\sqrt{\hat{v}_w^{(t)}}+\epsilon)}$$
  $$b^{(t)} = b^{(t-1)}-\alpha\cdot\frac{\hat{m}_b^{(t)}}{(\sqrt{\hat{v}_b^{(t)}}+\epsilon)}$$

where $\beta_1$ and $\beta_2$ are hyperparameters for the first and the second moment estimation

As we can see from the update rule, Adam dynamically forms boundaries for its learning rate using first and second moment estimate without requiring too much memory space. 

Note that you can reuse the `calculate_gradient` function in SGD for Adam. 

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
class Adam():
  def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7):
    """Initializes required parameters.

      Note: 
        self.m is a list of Tensor, representing first moment estimations of gradients. The initial value should be [0]. 
        self.v is a list of Tensor, representing second moment estimations of gradients. The initial value should be [0]. 
        self.t is the timestep. The initial value should be 0. 

      Args:
          lr (float): Learning rate.
          epsilon (float): Additive constant that ensures we do not divide by 0.
          beta1 (float): Hyperparameter for first moment estimation
          beta2 (float): Hyperparameter for second moment estimation
    """
    self.lr = lr
    self.beta1 = beta1
    self.beta2 = beta2
    self.epsilon = epsilon
    self.v = [0]
    self.m = [0]
    self.t = 0
    
    
  def calculate_gradient(self, X, y, model):
    """Calculates gradients in the computational graph.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          X (tf.Tensor): The input.
          y (tf.Tensor): The actual label.
          model (tf.keras.Model): The model used to predict y with given input X. 

      Returns:
          Tuple: (gradient w.r.t w, gradient w.r.t b)
    """

    N = len(X)

    diff = tf.subtract(model(X), y)
    grads_w =  2/N * tf.reduce_sum(tf.matmul(diff, X, transpose_a=True))
    grads_b =  2/N * tf.reduce_sum(diff)
    return (grads_w, grads_b)
    

  def apply_gradient(self, grads, model):
    """Updates parameters of the model.

      Note:
          You should try to make use of Tensorflow library to perform related calculation.

      Args:
          grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
          model (tf.keras.Model): The model used to predict y with given input X. 
    """
    # biased first moment estimate 
    self.m = tf.multiply(self.beta1, self.m) + tf.multiply((1-self.beta1), grads)
    self.v = tf.multiply(self.beta2, self.v) + tf.multiply((1-self.beta2), tf.square(grads))

    print("m", tf.shape(self.m))
    print("v", tf.shape(self.v))

    # normalize
    self.m = self.m / (1-self.beta1)
    self.v = self.v / (1-self.beta2)

    # update weights 
    model.w = model.w - self.lr * (self.m / (tf.sqrt(self.v) + self.epsilon))
    model.b = model.b - self.lr * (self.m / (tf.sqrt(self.v) + self.epsilon))


In [ ]:
# Evaluation
model_3 = Linear()
opt_3 = Adam(lr=0.1)
print("Initial loss: %.3f" % loss(model_3(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_3.calculate_gradient(X_train, y_train, model_3)
  # Use the optimizer to update gradients
  opt_3.apply_gradient(grads, model_3)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_3(X_train), y_train)))

print("Final loss: %.3f" % loss(model_3(X_train), y_train))
print("w = %s, b = %s" % (model_3.w.numpy(), model_3.b.numpy()))

w:  tf.Tensor([1 2], shape=(2,), dtype=int32)
x:  tf.Tensor([1 2], shape=(2,), dtype=int32)
y:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
b:  tf.Tensor([], shape=(0,), dtype=int32)
Initial loss: 68.725
w:  tf.Tensor([1 2], shape=(2,), dtype=int32)
x:  tf.Tensor([1 2], shape=(2,), dtype=int32)
y:  tf.Tensor([2000    1], shape=(2,), dtype=int32)
b:  tf.Tensor([], shape=(0,), dtype=int32)
m tf.Tensor([2], shape=(1,), dtype=int32)
v tf.Tensor([2], shape=(1,), dtype=int32)
w:  tf.Tensor([1 2], shape=(2,), dtype=int32)
x:  tf.Tensor([1 2], shape=(2,), dtype=int32)
y:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
b:  tf.Tensor([2], shape=(1,), dtype=int32)
Loss at epoch 0: 66.603
w:  tf.Tensor([1 2], shape=(2,), dtype=int32)
x:  tf.Tensor([1 2], shape=(2,), dtype=int32)
y:  tf.Tensor([2000    2], shape=(2,), dtype=int32)
b:  tf.Tensor([2], shape=(1,), dtype=int32)
m tf.Tensor([2], shape=(1,), dtype=int32)
v tf.Tensor([2], shape=(1,), dtype=int32)
w:  tf.Tensor([1 2], shape=(2,), dtyp

KeyboardInterrupt: ignored

In [ ]:
# Test case
assert np.allclose(loss(model_3(X_train), y_train), 1.0, atol=0.4), 'Wrong implementation of Adam'
